In [6]:
import {   bbsPlusGenerateSignatureParamsG1, bbsPlusGeneratePublicKeyG2, bbsPlusGenerateSigningKey, bbsPlusSignG1, bbsPlusVerifyG1, 
  bbsPlusInitializeProofOfKnowledgeOfSignature, bbsPlusGenProofOfKnowledgeOfSignature, bbsPlusVerifyProofOfKnowledgeOfSignature, 
  bbsPlusChallengeContributionFromProtocol, bbsPlusChallengeContributionFromProof, generateChallengeFromBytes, 
  initializeWasm
 } from 'crypto-wasm-new';
await initializeWasm();

In [7]:
const EID_DATA = {
    name: "Michael Jackson",
    age: 50,
    profession: "RSE"
}

In [8]:
const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

const messages = [
    Uint8Array.from(Buffer.from("message1", "utf8")),
    Uint8Array.from(Buffer.from("message2", "utf8")),
    Uint8Array.from(Buffer.from("message3", "utf8")),
];
const label = stringToBytes("test-params");
const messageCount = messages.length;

// Generate params deterministically using a label
const sigParams = bbsPlusGenerateSignatureParamsG1(messageCount, label);


In [9]:
// Generating KeyPair
const sk = bbsPlusGenerateSigningKey();
const pk = bbsPlusGeneratePublicKeyG2(sk, sigParams);

In [10]:
// Creating a signature
const signature = bbsPlusSignG1(messages, sk, sigParams, true);
console.log(`Signature in base 64: ${Buffer.from(signature).toString("base64")}`);

Signature in base 64: idlqlNE0xgfv50748LdK+9bYRxtP5CLvEhjFYn7BYbT0PsIvhomiHoV3/S/+FjVq0hlmthIc1r2PT6dVOpE17PL7T3MgMb0kLAQgdQep2Cdpe4DFukloOLulsgnkNDMpBe9QMZndqrS62pwtN6xiPA==


In [11]:
//Verify the signature
const isVerified = bbsPlusVerifyG1(messages, signature, pk, sigParams, true);
const isVerifiedString = JSON.stringify(isVerified);
console.log(`Signature verified ? ${isVerifiedString}`);

// Derive a proof from the signature revealing the first message
const revealed = new Set<number>();
revealed.add(0);
revealed.add(2);
const revealedMsgs = new Map();
revealedMsgs.set(0, messages[0]);
revealedMsgs.set(2, messages[2]);

const protocol = bbsPlusInitializeProofOfKnowledgeOfSignature(
    signature,
    sigParams,
    messages,
    new Map(),
    revealed,
    true
);
const challengeProver = generateChallengeFromBytes(
    bbsPlusChallengeContributionFromProtocol(protocol, revealedMsgs, sigParams, true)
);

const proof = bbsPlusGenProofOfKnowledgeOfSignature(protocol, challengeProver);

console.log(`Proof as base64: ${Buffer.from(proof).toString("base64")}`)

Signature verified ? {"verified":true}
Proof as base64: gqQ1vrQKumDtRfRoUw2qFW9/AWNdxIro2p2CzOu3MTwz7bBTHNR0GQlGUstYEBTHsJF5YgQB+NnBzRBL9T47OI9m7LtZ7zJx+f5DIB1eiH3c9FwhX49gcYA3s9QUukkNj1H5NZjy/uKskELFxw0bJXky40sNxHx9BGMswlellcVaYv/mjiigGV1eRHZ0HR6Btocl4iKciTb+04UvTQMXEVd5Ct9j/ZoNR2B/WSTU0caHBQMSWs9zGQhdiQl724YvusWs6w6gyC2zSUjXl7c9gdf1hElQoGLCKR/qAuSS6A2wmI+fwRwrKMahZcTc7NZwX0fO+tLAWplKncZ9TjjSM5lmZrxm08ZqJNnf+36DQlCoya2BT0ibPQSgc9RiiEvMl3QNgWAU5WZhDWrzBTkIbQMAAAAAAAAAPu7o8o1BXayGQDN/0OdjcrPpxpXfxvm2dHOhKKnQM2ghb31c1F05tHlaFd3Jx1w5YyyXhovniwN53eeKu9okagqm/+MPdz60XnesTjsy6GiQ7hhoWJVbrgbRg9Gw2Y0/


In [14]:
// Verify the created proof
const challengeVerifier = generateChallengeFromBytes(
    bbsPlusChallengeContributionFromProof(proof, revealedMsgs, sigParams, true)
);
const isProofVerified = bbsPlusVerifyProofOfKnowledgeOfSignature(
    proof, revealedMsgs, challengeVerifier, pk, sigParams, true
);
const isProofVerifiedString = JSON.stringify(isProofVerified);
console.log(`Proof Verified?: ${isProofVerifiedString}`);

Proof Verified?: {"verified":true}
